In [1]:
import numpy as np
import pandas as pd
import lightgbm as lgbm
import xgboost as xgb
import timeit

In [2]:
Train = pd.read_csv('mnist_train.csv',header=None)
Test = pd.read_csv('mnist_test.csv',header=None)

In [3]:
X_train = Train.values[:,1:]
X_test = Test.values[:,1:]
y_train = Train.values[:,0]
y_test = Test.values[:,0]

In [4]:
del Train,Test

In [5]:
X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)

LightGBM

In [46]:
params = {"objective": "multiclass",
          "boosting_type": "goss",
          "learning_rate": 1e-2,
          "num_leaves": 16,
          "max_bin": 63,
          "bagging_fraction" : 0.1,
          "feature_fraction": 0.3,
          "min_child_samples": 50,
          "min_child_weight": 50,
          "device_type": "gpu",
          "gpu_use_dp": False,
          "num_class" :10 # multiclass only
          }

In [7]:
dtrain = lgbm.Dataset(X_train, y_train)
dvalid = lgbm.Dataset(X_test, y_test, reference=dtrain)

GPU

In [47]:
start_time = timeit.default_timer()
model_lgbm = lgbm.train(params, dtrain, num_boost_round=1000, valid_sets=dvalid, verbose_eval=100,
                        early_stopping_rounds=10)
print("--- %s seconds ---" % (timeit.default_timer() - start_time))

Training until validation scores don't improve for 10 rounds.
[100]	valid_0's multi_logloss: 0.987472
[200]	valid_0's multi_logloss: 0.583752
[300]	valid_0's multi_logloss: 0.398319
[400]	valid_0's multi_logloss: 0.300727
[500]	valid_0's multi_logloss: 0.242835
[600]	valid_0's multi_logloss: 0.205347
[700]	valid_0's multi_logloss: 0.179255
[800]	valid_0's multi_logloss: 0.160742
[900]	valid_0's multi_logloss: 0.146893
[1000]	valid_0's multi_logloss: 0.136213
Did not meet early stopping. Best iteration is:
[1000]	valid_0's multi_logloss: 0.136213
--- 316.8527974949993 seconds ---


In [48]:
np.sum(y_test != model_lgbm.predict(X_test).argmax(1))/10000

0.0353

CPU

In [42]:
params["device_type"] = "cpu"
start_time = timeit.default_timer()
model_lgbm = lgbm.train(params, dtrain, num_boost_round=1000, valid_sets=dvalid, verbose_eval=100,
                        early_stopping_rounds=10)
print("--- %s seconds ---" % (timeit.default_timer() - start_time))

Training until validation scores don't improve for 10 rounds.
[100]	valid_0's multi_logloss: 0.98747
[200]	valid_0's multi_logloss: 0.583871
[300]	valid_0's multi_logloss: 0.398117
[400]	valid_0's multi_logloss: 0.300764
[500]	valid_0's multi_logloss: 0.242678
[600]	valid_0's multi_logloss: 0.205321
[700]	valid_0's multi_logloss: 0.179452
[800]	valid_0's multi_logloss: 0.160736
[900]	valid_0's multi_logloss: 0.146829
[1000]	valid_0's multi_logloss: 0.136007
Did not meet early stopping. Best iteration is:
[1000]	valid_0's multi_logloss: 0.136007
--- 70.99830526700043 seconds ---


In [43]:
np.sum(y_test != model_lgbm.predict(X_test).argmax(1))/10000

0.0353

Xgboost

In [55]:
dtrain = xgb.DMatrix(X_train,y_train)
dtest = xgb.DMatrix(X_test,y_test)

In [72]:
params =   {'objective':'multi:softprob',
            'eta': 1e-2,
            'max_leaves': 16, 
            'max_bin': 63,
            'subsample': 0.1,
            'colsample_bylevel': 0.3, 
            'min_child_weight':50,
            'grow_policy': 'lossguide',
            'tree_method':'gpu_hist',
            'num_class': 10,
            'eval_metric':'mlogloss'}

In [73]:
watchlist = [(dtest, 'eval'), (dtrain, 'train')]

GPU

In [74]:
start_time = timeit.default_timer()
model_gbm = xgb.train(params, dtrain, 1000, watchlist, early_stopping_rounds=10,verbose_eval=100)
print("--- %s seconds ---" % (timeit.default_timer() - start_time))

[0]	eval-mlogloss:2.284	train-mlogloss:2.28417
Multiple eval metrics have been passed: 'train-mlogloss' will be used for early stopping.

Will train until train-mlogloss hasn't improved in 10 rounds.
[100]	eval-mlogloss:1.19586	train-mlogloss:1.20975
[200]	eval-mlogloss:0.779445	train-mlogloss:0.79658
[300]	eval-mlogloss:0.575178	train-mlogloss:0.591336
[400]	eval-mlogloss:0.460818	train-mlogloss:0.475755
[500]	eval-mlogloss:0.392536	train-mlogloss:0.405376
[600]	eval-mlogloss:0.349175	train-mlogloss:0.360174
[700]	eval-mlogloss:0.320426	train-mlogloss:0.330032
[800]	eval-mlogloss:0.299829	train-mlogloss:0.308311
[900]	eval-mlogloss:0.284299	train-mlogloss:0.291714
[999]	eval-mlogloss:0.2723	train-mlogloss:0.278745
--- 119.5494857499998 seconds ---


In [66]:
np.sum(y_test != model_gbm.predict(dtest).argmax(1))/10000

0.0695

CPU

In [69]:
params['tree_method'] = 'hist'
params["silent"] = 1
params['objective'] = 'multi:softprob'
start_time = timeit.default_timer()
model_gbm = xgb.train(params, dtrain, 1000, watchlist, early_stopping_rounds=10,verbose_eval=100)
print("--- %s seconds ---" % (timeit.default_timer() - start_time))

[17:10:18] Tree method is selected to be 'hist', which uses a single updater grow_fast_histmaker.
[0]	eval-mlogloss:2.28351	train-mlogloss:2.28368
Multiple eval metrics have been passed: 'train-mlogloss' will be used for early stopping.

Will train until train-mlogloss hasn't improved in 10 rounds.
[100]	eval-mlogloss:1.17383	train-mlogloss:1.18571
[200]	eval-mlogloss:0.756954	train-mlogloss:0.772366
[300]	eval-mlogloss:0.553195	train-mlogloss:0.567739
[400]	eval-mlogloss:0.440835	train-mlogloss:0.453472
[500]	eval-mlogloss:0.374354	train-mlogloss:0.384899
[600]	eval-mlogloss:0.333485	train-mlogloss:0.342035
[700]	eval-mlogloss:0.305987	train-mlogloss:0.313222
[800]	eval-mlogloss:0.286282	train-mlogloss:0.292277
[900]	eval-mlogloss:0.271579	train-mlogloss:0.276535
[999]	eval-mlogloss:0.26033	train-mlogloss:0.264324
--- 199.22793187699972 seconds ---
